In [ ]:
from bs4 import BeautifulSoup as bs
import requests as req

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning
disable_warnings(InsecureRequestWarning)

import feedparser

In [ ]:

def rssfinder_function(url):

    print(" *** RSS Finder Function Started ! *** ")

    global used_url
    global finded_rss

    used_url = []
    finded_rss = []

    def Standard_Domain(url):

        try :

            # ***********************************************************
            # ***********************************************************

            def is_rss(url):

                print("'" + url + "' is Checking ..." )

                try :
                    feed = feedparser.parse(url)
                    feed_entries = feed.entries

                    if len(feed_entries) != 0:
                        return True
                    
                    else:
                        if req.get(url ).status_code == 200 :
                            res = req.get(url)
                            content_type = res.headers[ "Content-Type" ]
                            
                            if "application/rss+xml" in content_type :
                                return True

                            elif "application/xml" in content_type :
                                return True

                            elif "text/html" in content_type or "text/xml;" in content_type:
                                res = req.get(url)
                                soup = bs(res.text , "html.parser")

                                all_tags = soup.find_all()

                                if all_tags != None :
                                    for i in all_tags :
                                        if i.name == "rss" or i.name == "feed" or i.name == "feeds" or i.name == "channel" :
                                            return True

                        elif req.get(url ).status_code == 403 :
                            res = req.get(url)
                            content_type = res.headers[ "Content-Type" ]
                            
                            if "application/rss+xml" in content_type :
                                return True

                            elif "application/xml" in content_type :
                                return True

                            elif "text/html" in content_type or "text/xml;" in content_type:
                                res = req.get(url)
                                soup = bs(res.text , "html.parser")

                                all_tags = soup.find_all()

                                if all_tags != None :
                                    for i in all_tags :
                                        if i.name == "rss" or i.name == "feed" or i.name == "feeds" or i.name == "channel" :
                                            return True
                                            
                        else :
                            return False
                
                except :
                    return False

            # ***********************************************************
            # ***********************************************************

            def find_all_rss(url):
                if url[0] == "/" :
                    url = org_url + url

                try :

                    rss_link = []

                    res = req.get(url , verify=False)
                    soup = bs(res.text , "html.parser")

                    all_links = soup.find_all("a")

                    for i in all_links :
                        try :
                            if "rss" in i.get("href") or "RSS" in i.get("href") or "Rss" in i.get("href") or "feed" in i.get("href") or "FEED" in i.get("href") or "Feed" in i.get("href") :
                                rss_link.append(i.get("href"))

                        except :
                            continue

                    return rss_link

                except :
                    pass

            # ***********************************************************
            # ***********************************************************

            def main(url):

                used_url.append(url)

                if url[0] == "." :
                    url = url[1:]
                
                if url[0] == "/" :
                    url = org_url + url

                used_url.append(url)

                # print(url)

                # status = is_url_valid(url)

                try :

                    status = req.get(url).status_code

                    if status == 200 :

                        rss = is_rss(url)

                        if rss == True :

                            finded_rss.append(url)
                            # print( f"{url} is RSS" )

                        else :

                            all_links = find_all_rss(url)

                            if all_links != None :
                                for i in all_links :
                                    if i not in used_url :
                                        main(i)
                                    
                                    else :
                                        continue

                except :
                    pass

            # ***********************************************************
            # ***********************************************************

            def beta(url):

                try :

                    res = req.get(url)
                    soup = bs(res.text , "html.parser")

                    all_link = soup.find_all("a")

                    if len(all_link) != 0 :

                        for i in all_link:

                            try :

                                main(i.get("href"))
                            
                            except :
                                continue

                except :
                    pass

            # ***********************************************************
            # ***********************************************************

            def gama(url) :

                org = url

                dictionary = [ "feed" , "FEED" , "Feed" , "RSS" , "Rss" , "rss" , "fa/rss" , "fa/RSS" , "feeds" , "FEEDS" , "Feeds" , "FEEDs" , "RssForNews" ]

                for i in dictionary :

                    try:
                        pat_url = org + "/" + i
                        main(pat_url)

                    except :
                        continue


                if len(finded_rss) == 0:
                    if "www." in url or "WWW." in url or "Www." in url or "WWw." in url or "wWW." in url or "wwW." in url :
                        if "https://" in org and "http://" not in org:
                            for i in dictionary :
                                try :
                                    pat_url = url.replace( "https://www." , f"https://www.{i}." )

                                    if is_rss(pat_url) == True :
                                        finded_rss.append(pat_url)
                                
                                except :
                                    continue

                        elif "http://" in org and "https://" not in org :
                            for i in dictionary :

                                try :
                                    pat_url = url.replace( "http://www." , f"http://www.{i}." )

                                    if is_rss(pat_url) == True :
                                        finded_rss.append(pat_url)

                                except :
                                    continue

                    else :
                        if "https://" in org and "http://" not in org:
                            for i in dictionary :
                                try :
                                    pat_url = url.replace( "https://" , f"https://{i}." )

                                    if is_rss(pat_url) == True :
                                        finded_rss.append(pat_url)
                                
                                except :
                                    continue

                        elif "http://" in org and "https://" not in org :
                            for i in dictionary :

                                try :
                                    pat_url = url.replace( "http://" , f"http://{i}.{url}" )

                                    if is_rss(pat_url) == True :
                                        finded_rss.append(pat_url)

                                except :
                                    continue


            # ***********************************************************
            # ***********************************************************

            org_url = url
            main(url)

            if len(finded_rss) == 0 :
                beta(org_url)

            if len(finded_rss) == 0 :
                gama(org_url)

        except :
            pass

# =========================================================================================
# =========================================================================================

    def unStandard_Domain(url):

        try :
            # ***********************************************************
            # ***********************************************************

            def use_selenium (url):

                try :

                    # print(" 'use_selenium' function() ")

                    options = webdriver.ChromeOptions()
                    options.add_argument('headless')
                    driver = webdriver.Chrome(ChromeDriverManager().install() , chrome_options=options)
                    driver.get(url)
                    html = driver.page_source
                    soup = bs(html)

                    return soup
                
                except :
                    pass

            # ***********************************************************
            # ***********************************************************

            def is_rss_selenium(url):

                print("'" + url + "' is Checking ..." )

                # print(" 'is_rss_selenium' function() ")

                try :
                    feed = feedparser.parse(url)
                    feed_entries = feed.entries

                    if len(feed_entries) != 0:
                        return True

                    else :

                        soup = use_selenium(url)

                        all_tags = soup.find_all()

                        if all_tags != None :
                            for i in all_tags :
                                if i.name == "rss" or i.name == "feed" or i.name == "feeds" or i.name == "channel" :
                                    return True
                
                except :
                    return False

            # ***********************************************************
            # ***********************************************************

            def find_all_rss_selenium(url):

                # print(" 'find_all_rss_selenium' function() ")

                if url[0] == "/" :
                    url = org_url + url

                try :

                    rss_link = []
                    soup = use_selenium(url)
                    all_links = soup.find_all("a")

                    for i in all_links :
                        try :
                            if "rss" in i.get("href") or "RSS" in i.get("href") or "Rss" in i.get("href") or "feed" in i.get("href") or "FEED" in i.get("href") or "Feed" in i.get("href") :
                                rss_link.append(i.get("href"))

                        except :
                            continue

                    return rss_link

                except :
                    pass

            # ***********************************************************
            # ***********************************************************

            def main_selenium(url):

                try :

                    # print(" 'main_selenium' function() ")

                    used_url.append(url)

                    if url[0] == "." :
                        url = url[1:]
                    
                    if url[0] == "/" :
                        url = org_url + url

                    used_url.append(url)

                    # print(url)

                    rss = is_rss_selenium(url)

                    if rss == True :

                        finded_rss.append(url)
                        # print( f"{url} is RSS" )

                    else :

                        all_links = find_all_rss_selenium(url)

                        if all_links != None :
                            for i in all_links :
                                try :
                                    if i not in used_url :
                                        main_selenium(i)
                                    
                                    else :
                                        continue

                                except:
                                    continue
                except:
                    pass

            # ***********************************************************
            # ***********************************************************

            def beta_selenium(url):

                try :

                    soup = use_selenium(url)

                    all_link = soup.find_all("a")

                    if len(all_link) != 0 :

                        for i in all_link:

                            try:
                                main_selenium(i.get("href"))
                            except:
                                continue

                except:
                    pass

            # ***********************************************************
            # ***********************************************************

            def gama_selenium(url) :

                org = url

                dictionary = [ "feed" , "FEED" , "Feed" , "RSS" , "Rss" , "rss" , "fa/rss" , "fa/RSS" , "feeds" , "FEEDS" , "Feeds" , "FEEDs" , "RrrForNews" ]

                for i in dictionary :

                    try:      
                        pat_url = org + "/" + i
                        main_selenium(pat_url)

                    except:
                        continue

                if len(finded_rss) == 0:

                    if "www." in url or "WWW." in url or "Www." in url or "WWw." in url or "wWW." in url or "wwW." in url :
                        if "https://" in org and "http://" not in org:
                            for i in dictionary :
                                try:
                                    pat_url = url.replace( "https://www." , f"https://www.{i}." )
                                    if is_rss_selenium(pat_url) == True :
                                        finded_rss.append(pat_url)
                                except:
                                    continue

                        elif "http://" in org and "https://" not in org :
                            for i in dictionary :
                                try :  
                                    pat_url = url.replace( "http://" , f"http://www.{i}." )
                                    if is_rss_selenium(pat_url) == True :
                                        finded_rss.append(pat_url)
                                except:
                                    continue

                    else :
                        if "https://" in org and "http://" not in org:
                            for i in dictionary :
                                try:
                                    pat_url = url.replace( "https://" , f"https://{i}." )
                                    if is_rss_selenium(pat_url) == True :
                                        finded_rss.append(pat_url)
                                except:
                                    continue

                        elif "http://" in org and "https://" not in org :
                            for i in dictionary :
                                try :  
                                    pat_url = url.replace( "http://" , f"http://{i}.{url}" )
                                    if is_rss_selenium(pat_url) == True :
                                        finded_rss.append(pat_url)
                                except:
                                    continue


            # ***********************************************************
            # ***********************************************************
            
            org_url = url
            main_selenium(url)
            

            print("befor beta")
            print("len findedrss : " , len(finded_rss))

            if len(finded_rss) == 0 :
                beta_selenium(org_url)


            print("befor gama")
            print("len findedrss : " , len(finded_rss))

            if len(finded_rss) == 0 :
                gama_selenium(org_url)

        except:
            pass


    def pre_set (url) :

        urls = []

        if "https" in url :
            urls.append(url)
            url = url.replace( "https" , "http" )
            urls.append(url)

        elif "https" not in url and "http" in url :

            url = url.replace( "http" , "https" )
            urls.append(url)
            url = url.replace( "https" , "http" )
            urls.append(url)

        elif "https" not in url and "http" not in url :

            url_01 = "https://" + url
            urls.append(url_01)

            url_02 = "http://" + url
            urls.append(url_02)

        return urls

# =========================================================================================
# =========================================================================================

    urls = pre_set(url)

    # 00
    try :

        status = req.get(urls[0]).status_code

        if status == 200 :

            Standard_Domain(urls[0])

            if len(finded_rss) == 0 :

                Standard_Domain(urls[1])

        
        elif status == 403 or status == 500 :

            unStandard_Domain(urls[0])

            if len(finded_rss) == 0 :

                unStandard_Domain(urls[1])


    except :

        unStandard_Domain(urls[0])

        if len(finded_rss) == 0 :

            unStandard_Domain(urls[1])

    
    print(" *** RSS Finder Function Finished ! *** ")
    return finded_rss



In [ ]:
rssfinder_function(url)